### Business Case Study - Staff Planning

#### Problem Statement

The insurance approval process, i.e., underwriting is one of the important and time-consuming tasks in an insurance application processing. When you submit your insurance application, the underwriter of the company evaluates it based on the details that you provide using a rule-based or an ML model and decides whether or not to approve your application.

An insurance company InsurePlus wants you to help them with finding the optimal number of staff that they need for their insurance application approval process for the calendar year 2021. In the industry, the number of staffs is considered as a continuous variable. This is also called a Full-Time Equivalent (FTE) of the staff. 

The company operates in three states: A, B and C. The company can either handle an application with the staff that they hire or outsource it to a vendor. Assume that there is no capacity limitation to outsourcing. If they hire staff, he/she can handle 40 insurance applications per month when he/she works 100% of the workdays. However, there are days that he/she will be unavailable to process applications due to training, off days, etc. A staff member’s availability (in percentage) to work on processing the insurance applications for each month is shown in the table given below. As mentioned before, with 100% availability, each member can handle 40 applications. A special note of practical relevance: In the industry, staff availability is predicted using a time-motion study. But in this case, you have been given fixed numbers for each month in the table above. You can read more about the time-motion study here if you are curious but please note that it is not required to solve the case study.

The objective is to optimise the total cost for the application approval process by distributing the right number of applications between the FTEs and the vendors while meeting the monthly demand for each state at the same time

In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from Excel workbook - demand tab
demand = pd.read_excel("Staffing+Data.xlsx",sheet_name='DemandData')

# Reading the data from Excel workbook - staff availablity tab
staff_data = pd.read_excel("Staffing+Data.xlsx",sheet_name='StaffAvailability')

# Reading the data from Excel workbook - Cost
cost = pd.read_excel("Staffing+Data.xlsx",sheet_name='Cost')

# Rate of applications that 1 FTE can cater to in a month @ 100% efficiency
srv_rate = 40


In [4]:
# Viewing the exported data
demand.head()

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942
3,A,Apr,2297
4,A,May,1992


In [5]:
# Viewing the exported data
staff_data.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [6]:
# Viewing the exported data
cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180
3,A,Apr,60000,5000.0,180
4,A,May,60000,5000.0,180


**`COMMENTS :`**<br/>State & Month are the common columns across all 3 dataframes 

## Data pre-processing 

In [7]:
# Creating unique index values from the dataset
month_name = demand['Month'].unique()
state_name = demand['State'].unique()
print("state :",state_name)
print("months :",month_name)

state : ['A' 'B' 'C']
months : ['Jan' 'Feb' 'Mar' 'Apr' 'May' 'Jun' 'Jul' 'Aug' 'Sep' 'Oct' 'Nov' 'Dec']


In [8]:
# Creating dict for the required parameters with [State, Month] as indexes

#Application Demand across State & Month
Demand_data = demand.set_index(['State','Month'])['Demand'].to_dict()

# Cost per Application for Outsource 
OutsourceRate = cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Cost for FTE monthly salary 
StaffMsal = cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

# Staff availabilty for serving the Insurance - regular scenario , Lower bound , Upper bound scenario
StaffAvPer_data = staff_data.set_index(['State','Month'])['StaffAvPer'].to_dict()
StaffLB_data = staff_data.set_index(['State','Month'])['LB'].to_dict()
StaffUB_data = staff_data.set_index(['State','Month'])['UB'].to_dict()

Staff_app_permonth = 40


# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



<div class="alert alert-block alert-success">
    
**`ANSWER to Q1`**<br/>

#### Case Study Objective :
The objective of the case study is to identify the optimal number of staff between Employees & Outsource such as to minimize the total cost for application approval process

#### Decision Variables :
The objective indicates towards requirement of Cost & Count for developing the mathematical equation of the Objective Function.
For Outsource - the cost is provided at per application basis & hence the $1st Variable$ will be **No. of Applications from Outsource named - `Outsource_appl`** across states & months
For Employess - the cost is provided as monthly salary & hence the $2nd Variable$ will be **No. of FTE named - `FTE_count`** across states & months 

#### Objective Function :
To minimize the total cost for application approval process the Objective function will be : <br/>
$ \sum_{i \in S}\sum_{j \in M} (c_{ij} x_{ij} + O_{ij} y_{ij}) \,\,\,\,\,\,\,\,\, $ <br/>

$ \forall i \in S \,\,\,\,\,\, \forall j \in M$ <br/>

Subject To: <br>
$ x_{ij}*40*a_{ij} + y_{ij} \leq D_{ij} \,\,\,\,\,\,\,\,\, \forall i \in S \,\,\,\,\,\, \forall j \in M$ 

where,
- S = States
- M = Months
- c = Staff monthly salary
- x = no. of FTE 
- O = Outsource Cost
- y = No. of applications by outsource
- a = Staff Availability %
- D = Application Demand 
- 40 = No. of Application per month by staff incase of 100% availability


**[Constraints](#Constraints)**

#### 1. Demand per month per state
Before we create the Demand Constraint, we need to calculate a variable for No. of Applications being processed by FTEs.<br/>
    - FTE_Appl = No. of FTE * StaffAvPer_data * Staff_app_permonth 
**Demand_Constraint** = FTE_Appl + Outsource_appl _for each month for each state_

$ (x_{ij}*40*a_{ij})+ y_{ij}\,\,=\,\,D_{ij} \,\,\,\,\,\,\,\,\, \forall i \in S \,\,\,\,\,\, \forall j \in M$ 

#### 2. Maximum Outsource for State A
State A can only provide 30% of the total application demand to outsource <br/>
$ y_{ij}\,\leq\,D_{ij}*0.3 \,\,\,\,\,\,\,\,\, \forall j \in M \,\,\,\,\,\, \forall i \in stateA  $

#### 3. Maximum Outsource for State B
State B can only provide 40% of the total application demand to outsource <br/>
$ y_{ij}\,\leq\,D_{ij}*0.4 \,\,\,\,\,\,\,\,\, \forall j \in M \,\,\,\,\,\, \forall i \in stateB  $


**Note:** mathematical formulation provided in report too.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [9]:
# Creating a model instance
model = ConcreteModel()

<div class="alert alert-block alert-success">

**Pyomo sets and Parameters**

In [10]:
# Initializing month & state for usage in loops ahead
model.location = Set(initialize = state_name.tolist(),doc = 'States')
model.months = Set(initialize=month_name.tolist(), doc = 'Months')

In [11]:
model.location.display()

location : States
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    3 : {'A', 'B', 'C'}


**Parameter 1 : Demand**

In [12]:
# Defining model Parameter for Demand across months & states
model.d = Param(model.location, model.months, initialize = Demand_data, doc = 'Demand')
model.d.display()

d : Demand
    Size=36, Index=d_index, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('A', 'Apr') :  2297
    ('A', 'Aug') :  3371
    ('A', 'Dec') :  5183
    ('A', 'Feb') :  4878
    ('A', 'Jan') :  5240
    ('A', 'Jul') :  5334
    ('A', 'Jun') :  2275
    ('A', 'Mar') :  5942
    ('A', 'May') :  1992
    ('A', 'Nov') :  4284
    ('A', 'Oct') :  3529
    ('A', 'Sep') :  3759
    ('B', 'Apr') :  2338
    ('B', 'Aug') :  2620
    ('B', 'Dec') :  4227
    ('B', 'Feb') :  2628
    ('B', 'Jan') :  4927
    ('B', 'Jul') :  4271
    ('B', 'Jun') :  3147
    ('B', 'Mar') :  2974
    ('B', 'May') :  4020
    ('B', 'Nov') :  3137
    ('B', 'Oct') :  4155
    ('B', 'Sep') :  4517
    ('C', 'Apr') :  2261
    ('C', 'Aug') :  2496
    ('C', 'Dec') :  1998
    ('C', 'Feb') :  1967
    ('C', 'Jan') :  1162
    ('C', 'Jul') :  2489
    ('C', 'Jun') :  1642
    ('C', 'Mar') :  1898
    ('C', 'May') :  2030
    ('C', 'Nov') :   963
    ('C', 'Oct') :  2421
    ('C', 'Sep') :   

**Parameter 2 : Staff Availability**

In [13]:
# Defining model Parameter for Staff Availability across months & states
model.avail = Param(model.location, model.months, initialize = StaffAvPer_data, doc = 'Staff_Availability')
model.avail.display()

avail : Staff_Availability
    Size=36, Index=avail_index, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('A', 'Apr') :   0.8
    ('A', 'Aug') :  0.76
    ('A', 'Dec') :  0.65
    ('A', 'Feb') :  0.76
    ('A', 'Jan') :  0.81
    ('A', 'Jul') :  0.68
    ('A', 'Jun') :  0.73
    ('A', 'Mar') :  0.75
    ('A', 'May') :  0.78
    ('A', 'Nov') :  0.68
    ('A', 'Oct') :  0.73
    ('A', 'Sep') :  0.81
    ('B', 'Apr') :   0.8
    ('B', 'Aug') :  0.76
    ('B', 'Dec') :  0.65
    ('B', 'Feb') :  0.76
    ('B', 'Jan') :  0.81
    ('B', 'Jul') :  0.68
    ('B', 'Jun') :  0.73
    ('B', 'Mar') :  0.75
    ('B', 'May') :  0.78
    ('B', 'Nov') :  0.68
    ('B', 'Oct') :  0.73
    ('B', 'Sep') :  0.81
    ('C', 'Apr') :   0.8
    ('C', 'Aug') :  0.76
    ('C', 'Dec') :  0.65
    ('C', 'Feb') :  0.76
    ('C', 'Jan') :  0.81
    ('C', 'Jul') :  0.68
    ('C', 'Jun') :  0.73
    ('C', 'Mar') :  0.75
    ('C', 'May') :  0.78
    ('C', 'Nov') :  0.68
    ('C', 'Oct') :  0.73
 

**Parameter 3 : Staff Monthly Salary**

In [14]:
# Defining model Parameter for Staff Monthly Salary across months & states
model.staffsalary = Param(model.location,model.months, initialize=StaffMsal, doc='Staff Monthly Salary')
model.staffsalary.display()

staffsalary : Staff Monthly Salary
    Size=36, Index=staffsalary_index, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('A', 'Apr') :            5000.0
    ('A', 'Aug') :            5000.0
    ('A', 'Dec') :            5000.0
    ('A', 'Feb') :            5000.0
    ('A', 'Jan') :            5000.0
    ('A', 'Jul') :            5000.0
    ('A', 'Jun') :            5000.0
    ('A', 'Mar') :            5000.0
    ('A', 'May') :            5000.0
    ('A', 'Nov') :            5000.0
    ('A', 'Oct') :            5000.0
    ('A', 'Sep') :            5000.0
    ('B', 'Apr') : 4583.333333333333
    ('B', 'Aug') : 4583.333333333333
    ('B', 'Dec') : 4583.333333333333
    ('B', 'Feb') : 4583.333333333333
    ('B', 'Jan') : 4583.333333333333
    ('B', 'Jul') : 4583.333333333333
    ('B', 'Jun') : 4583.333333333333
    ('B', 'Mar') : 4583.333333333333
    ('B', 'May') : 4583.333333333333
    ('B', 'Nov') : 4583.333333333333
    ('B', 'Oct') : 4583.333333333333
    ('B', '

**Parameter 4 : Outsource Cost**

In [15]:
# Defining model Parameter for Outsource Cost across months & states
model.outsourcecost = Param(model.location, model.months, initialize = OutsourceRate, doc = 'Outsource Cost')
model.outsourcecost.display()

outsourcecost : Outsource Cost
    Size=36, Index=outsourcecost_index, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('A', 'Apr') :   180
    ('A', 'Aug') :   180
    ('A', 'Dec') :   180
    ('A', 'Feb') :   180
    ('A', 'Jan') :   180
    ('A', 'Jul') :   180
    ('A', 'Jun') :   180
    ('A', 'Mar') :   180
    ('A', 'May') :   180
    ('A', 'Nov') :   180
    ('A', 'Oct') :   180
    ('A', 'Sep') :   180
    ('B', 'Apr') :   150
    ('B', 'Aug') :   150
    ('B', 'Dec') :   150
    ('B', 'Feb') :   150
    ('B', 'Jan') :   150
    ('B', 'Jul') :   150
    ('B', 'Jun') :   150
    ('B', 'Mar') :   150
    ('B', 'May') :   150
    ('B', 'Nov') :   150
    ('B', 'Oct') :   150
    ('B', 'Sep') :   150
    ('C', 'Apr') :   160
    ('C', 'Aug') :   160
    ('C', 'Dec') :   160
    ('C', 'Feb') :   160
    ('C', 'Jan') :   160
    ('C', 'Jul') :   160
    ('C', 'Jun') :   160
    ('C', 'Mar') :   160
    ('C', 'May') :   160
    ('C', 'Nov') :   160
    ('C', 'Oct

<div class="alert alert-block alert-success">

**Decision Variables**

In [16]:
# Decision variables
model.x = Var(model.location, model.months, domain=NonNegativeReals, doc='No. of FTE')
model.y = Var(model.location, model.months, domain=NonNegativeIntegers, doc='No. of Outsource Appl.')

<div class="alert alert-block alert-success">

**Objective Function**

In [17]:
obj = 0
def obj_rule(model):
    global obj
    for l in model.location:
        for m in model.months:
            # formula used is : (Outsource Appl * per Appl cost for Outsource) + (no. of FTEs * Monthly Salary)
            obj+=(model.y[l,m]*model.outsourcecost[l,m])+(model.x[l,m]*model.staffsalary[l,m])
    return obj
model.output = Objective(rule=obj_rule, sense= minimize)

<div class="alert alert-block alert-success">
<a id="Constraints"></a>
    
**Constraints**

**Constraint 1 : Demand Constraint**

In [18]:
model.ddConst = ConstraintList()
for l in model.location:
    for m in model.months:
        model.ddConst.add(expr = model.y[l,m] + model.x[l,m]*(model.avail[l,m]*40) == model.d[l,m]) 
        # Demand = No. of Outsource Applications + (FTE count* Max Application,i.e.,'40' * Staff Availability %)

**Constraint 2 : State A Outsourcing Constraint**

In [19]:
model.statelimit = ConstraintList()

for l in model.location:
    for m in model.months:
        if l == 'A':
            model.statelimit.add(expr = model.y[l,m] <= 0.3*model.d[l,m])
        elif l == 'B':
            model.statelimit.add(expr = model.y[l,m] <= 0.4*model.d[l,m])
        else:
            pass

<div class="alert alert-block alert-success">

**Invoking Solver**

In [20]:
# Invoking the solver
result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.20064353942871094
# ----------------------------------------------------------
#   Solution Information
# ------------

In [21]:
# Print the value of the objective function (converting the output to million)
base_cost = int(model.output())/10**6
base_cost

17.962336

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [22]:
# Creating dataframe for the results
# optimal number of staff members for the Avg case scenario
Avg_Output = []
Avg_avilb = 0

for l in model.location:
    for m in model.months:
        
        # optimal no. of staff members for Avg Case scenario
        Staff_count = (round(model.x[l,m].value))
        
        # outsourced applications% = no. of Outsource applications(stored in variable y) divide by total demand(stored in variable d)*100
        Outsource_applications = (model.y[l,m].value)
        
        # Total Cost = cost of staff + Cost of   outsourced  applications
                
        Avg_Output.append([l,m, Staff_count, Outsource_applications])
       
                
print(Avg_Output)


[['A', 'Jan', 162, 0.0], ['A', 'Feb', 160, 0.0], ['A', 'Mar', 198, 0.0], ['A', 'Apr', 72, 0.0], ['A', 'May', 64, 0.0], ['A', 'Jun', 78, 0.0], ['A', 'Jul', 137, 1600.0], ['A', 'Aug', 111, 0.0], ['A', 'Sep', 116, 0.0], ['A', 'Oct', 121, 0.0], ['A', 'Nov', 110, 1285.0], ['A', 'Dec', 140, 1554.0], ['B', 'Jan', 152, 0.0], ['B', 'Feb', 52, 1051.0], ['B', 'Mar', 60, 1189.0], ['B', 'Apr', 73, 0.0], ['B', 'May', 129, 0.0], ['B', 'Jun', 65, 1258.0], ['B', 'Jul', 94, 1708.0], ['B', 'Aug', 52, 1048.0], ['B', 'Sep', 139, 0.0], ['B', 'Oct', 85, 1662.0], ['B', 'Nov', 69, 1254.0], ['B', 'Dec', 98, 1690.0], ['C', 'Jan', 36, 0.0], ['C', 'Feb', 65, 0.0], ['C', 'Mar', 63, 0.0], ['C', 'Apr', 71, 0.0], ['C', 'May', 65, 0.0], ['C', 'Jun', 56, 0.0], ['C', 'Jul', 0, 2489.0], ['C', 'Aug', 82, 0.0], ['C', 'Sep', 28, 0.0], ['C', 'Oct', 83, 0.0], ['C', 'Nov', 0, 963.0], ['C', 'Dec', 0, 1998.0]]


In [23]:
Avg_Output = pd.DataFrame(Avg_Output, columns=['State','Month','Optimal Staff_AvgCase','Out_Appl_AvgCase'])
Avg_Output

,State,Month,Optimal Staff_AvgCase,Out_Appl_AvgCase
0,A,Jan,162,0.0
1,A,Feb,160,0.0
2,A,Mar,198,0.0
3,A,Apr,72,0.0
4,A,May,64,0.0
5,A,Jun,78,0.0
6,A,Jul,137,1600.0
7,A,Aug,111,0.0
8,A,Sep,116,0.0
9,A,Oct,121,0.0


In [24]:
# Importing the required library
from openpyxl import load_workbook

In [25]:
# Writing the results in to an Excel sheet
# Writing the output data into an Excel sheet named "Staffing_Data.xlsx" workbook


book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output'
Avg_Output.to_excel(writer, sheet_name='Output_Staff_Planning_Avg')



# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


In [26]:
# creating dataframe to record the Overall output of Average Scenario for further scenario

from statistics import mean
final_output = []
appl_cost = 0
out_appl = 0
for l in model.location:
    for m in model.months:
            
            cost_per_appl=((model.y[l,m].value*model.outsourcecost[l,m])+(model.x[l,m].value*model.staffsalary[l,m]))/model.d[l,m]
            appl_cost+=cost_per_appl
            final_output.append(cost_per_appl)
            out_appl+= model.y[l,m].value
            
base_mean = mean(final_output)
overall_df = pd.DataFrame(data={(round(base_mean,1),round(appl_cost,1),round(base_cost,2),round(out_appl,))},index=['Base Scenario'],columns=['Avg_cost_per_appl','tot_cost_per_appl','Overall Cost_in $mio','Outsource_Appl_count'])
overall_df

,Avg_cost_per_appl,tot_cost_per_appl,Overall Cost_in $mio,Outsource_Appl_count
Base Scenario,155.9,5611.7,17.96,20749


# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

<div class="alert alert-block alert-success">
    
### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [27]:
# Creating a model instance
model_lb = ConcreteModel()

**`COMMENTS :`**All sets & parameters remain same except availability as the Staff Availability % is a different reference for worst case scenario. Recreating all for Lower Bound model :  model_lb 

In [28]:
# displaying the location set
model_lb.location = Set(initialize = state_name.tolist(),doc = 'States')
model_lb.location.display()

location : States
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    3 : {'A', 'B', 'C'}


In [29]:
# displaying the months set
model_lb.months = Set(initialize=month_name.tolist(), doc = 'Months')
model_lb.months.display()

months : Months
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   12 : {'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'}


**`COMMENTS :`** No change in Parameter logics for Demand , Staff Monthly Salary & Outsource Cost. Recreating the same for Lower Bound model :  model_lb 

In [30]:
model_lb.d = Param(model_lb.location, model_lb.months, initialize = Demand_data, doc = 'LB Demand')
model_lb.staffsalary = Param(model_lb.location,model_lb.months, initialize=StaffMsal, doc='LB Staff Monthly Salary')
model_lb.outsourcecost = Param(model_lb.location, model_lb.months, initialize = OutsourceRate, doc = 'LB Outsource Cost')

In [31]:
# Recalculating the Availability basis LB% 
LB_data = staff_data.set_index(['State','Month'])['LB'].to_dict()
model_lb.LB = Param(model_lb.location, model_lb.months, initialize = LB_data, doc = 'Lower Bound on Availability')
model_lb.LB.display()

LB : Lower Bound on Availability
    Size=36, Index=LB_index, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('A', 'Apr') :  0.75
    ('A', 'Aug') :  0.65
    ('A', 'Dec') :   0.6
    ('A', 'Feb') :  0.65
    ('A', 'Jan') :   0.7
    ('A', 'Jul') :   0.6
    ('A', 'Jun') :  0.65
    ('A', 'Mar') :   0.7
    ('A', 'May') :   0.7
    ('A', 'Nov') :   0.6
    ('A', 'Oct') :  0.65
    ('A', 'Sep') :   0.7
    ('B', 'Apr') :  0.75
    ('B', 'Aug') :  0.65
    ('B', 'Dec') :   0.6
    ('B', 'Feb') :  0.65
    ('B', 'Jan') :   0.7
    ('B', 'Jul') :   0.6
    ('B', 'Jun') :  0.65
    ('B', 'Mar') :   0.7
    ('B', 'May') :   0.7
    ('B', 'Nov') :   0.6
    ('B', 'Oct') :  0.65
    ('B', 'Sep') :   0.7
    ('C', 'Apr') :  0.75
    ('C', 'Aug') :  0.65
    ('C', 'Dec') :   0.6
    ('C', 'Feb') :  0.65
    ('C', 'Jan') :   0.7
    ('C', 'Jul') :   0.6
    ('C', 'Jun') :  0.65
    ('C', 'Mar') :   0.7
    ('C', 'May') :   0.7
    ('C', 'Nov') :   0.6
    ('C', 'Oct') :  0.6

In [32]:
# Decision variables
model_lb.x = Var(model_lb.location, model_lb.months, domain=NonNegativeReals, doc='No. of FTE for LB')
model_lb.y = Var(model_lb.location, model_lb.months, domain=NonNegativeIntegers, doc='No. of Outsource Appl. for LB')

In [33]:
# Constraints
# Demand Constraint gets changed as the Availability % is to be replaced by LB % values. 

model_lb.dConst = ConstraintList()

for l in model_lb.location:
    for m in model_lb.months:
        model_lb.dConst.add(expr = model_lb.y[l,m] + model_lb.x[l,m]*(model_lb.LB[l,m]*40) == model_lb.d[l,m]) 

In [34]:
# No change in the logic for Outsourcing constraint by State A & B. Recreating the constraint for model_lb
model_lb.statelimit = ConstraintList()

for l in model_lb.location:
    for m in model_lb.months:
        if l == 'A':
            model_lb.statelimit.add(expr = model_lb.y[l,m] <= 0.3*model_lb.d[l,m])
        elif l == 'B':
            model_lb.statelimit.add(expr = model_lb.y[l,m] <= 0.4*model_lb.d[l,m])
        else:
            pass

In [35]:
# Objective function - the function is similar as in base scenario : 
# (per Appl cost for Outsource*Outsource Appl) + (Monthly Salary * no. of FTEs)
obj = 0
def obj_rule_lb(model_lb):
    global obj
    for l in model_lb.location:
        for m in model_lb.months:
            obj+=(model_lb.y[l,m]*model_lb.outsourcecost[l,m])+(model_lb.x[l,m]*model_lb.staffsalary[l,m])
    return obj

model_lb.result = Objective(rule=obj_rule_lb, sense= minimize)

In [36]:
# Invoking the solver
result_lb = SolverFactory('glpk').solve(model_lb)
result_lb.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.07813119888305664
# ----------------------------------------------------------
#   Solution Information
# ------------

In [37]:
# Print the value of the objective function
worst_case_cost = int(model_lb.result())/10**6
worst_case_cost

19.599482

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

#### 3.1.1 Optimal number of staff members 

In [38]:
# optimal number of staff members for the worst case scenario
lb_staff_count = []
lb_avilb = 0

for l in model_lb.location:
    for m in model_lb.months:
        
        # optimal no. of staff members for Worst Case scenario
        lb_staff_count.append([l,m,round(model_lb.x[l,m].value,)])
        
lb_staff_count = pd.DataFrame(lb_staff_count, columns=['State','Month','Optimal Staff_WorstCase'])
lb_staff_count


,State,Month,Optimal Staff_WorstCase
0,A,Jan,187
1,A,Feb,131
2,A,Mar,212
3,A,Apr,77
4,A,May,71
5,A,Jun,61
6,A,Jul,156
7,A,Aug,91
8,A,Sep,134
9,A,Oct,95


#### 3.1.2 Percentage of outsourced applications 

In [39]:
# write your code here
out_appl = []
for l in model_lb.location:
    for m in model_lb.months:
        
        # outsourced applications% = no. of Outsource applications(stored in variable y) divide by total demand(stored in variable d)*100
        outsource_percent = round(((model_lb.y[l,m].value)/(model_lb.d[l,m]))*100,1)  
        out_appl.append([l,m,outsource_percent])

out_appl = pd.DataFrame(out_appl, columns=['State','Month','Outsource Application %'])
out_appl #percentage of outsourced applications for Worst Case scenario

,State,Month,Outsource Application %
0,A,Jan,0.0
1,A,Feb,30.0
2,A,Mar,0.0
3,A,Apr,0.0
4,A,May,0.0
5,A,Jun,30.0
6,A,Jul,30.0
7,A,Aug,30.0
8,A,Sep,0.0
9,A,Oct,30.0


In [40]:
# Overall % of outsourced Applications 
out = 0
dd = 0

for l in model_lb.location:
    for m in model_lb.months:
        out+=(model_lb.y[l,m].value)
        dd+=(model_lb.d[l,m])
        
LB_appl_per = round(out/dd*100,1)
print("outsource Appl :",out)
print("total demand :",dd)
print("Overall percentage of Outsource Applications in Worst Case Scenario is :",LB_appl_per,"%")

outsource Appl : 39808.0
total demand : 113294
Overall percentage of Outsource Applications in Worst Case Scenario is : 35.1 %


#### 3.1.3 Average cost per application

In [41]:
# write your code here

from statistics import mean 
cost_per_appl = []

for l in model_lb.location:
    for m in model_lb.months:
        appl_cost = ((model_lb.y[l,m].value*model_lb.outsourcecost[l,m])+(model_lb.x[l,m].value*model_lb.staffsalary[l,m]))/model_lb.d[l,m]
        cost_per_appl.append(appl_cost)

cost = mean(cost_per_appl)
print("Average Cost per application for Worst Case scenario :",round(cost,2))

Average Cost per application for Worst Case scenario : 169.57


#### Wriiting output to an excel for Worst Case

In [42]:
# Creating dataframe for the results
# optimal number of staff members for the Worst case scenario
LB_Output = []
LB_avilb = 0

for l in model_lb.location:
    for m in model_lb.months:
        
        # optimal no. of staff members for Worst Case scenario
        Staff_count_LB = (round(model_lb.x[l,m].value))
        
        # outsourced applications% = no. of Outsource applications(stored in variable y) divide by total demand(stored in variable d)*100
        Outsource_applications_LB = (model_lb.y[l,m].value)
        
                       
        LB_Output.append([l,m, Staff_count_LB, Outsource_applications_LB])
       
                
print(LB_Output)


[['A', 'Jan', 187, 0.0], ['A', 'Feb', 131, 1463.0], ['A', 'Mar', 212, 0.0], ['A', 'Apr', 77, 0.0], ['A', 'May', 71, 0.0], ['A', 'Jun', 61, 682.0], ['A', 'Jul', 156, 1600.0], ['A', 'Aug', 91, 1011.0], ['A', 'Sep', 134, 0.0], ['A', 'Oct', 95, 1058.0], ['A', 'Nov', 125, 1285.0], ['A', 'Dec', 151, 1554.0], ['B', 'Jan', 106, 1970.0], ['B', 'Feb', 61, 1051.0], ['B', 'Mar', 64, 1189.0], ['B', 'Apr', 47, 935.0], ['B', 'May', 86, 1608.0], ['B', 'Jun', 73, 1258.0], ['B', 'Jul', 107, 1708.0], ['B', 'Aug', 60, 1048.0], ['B', 'Sep', 97, 1806.0], ['B', 'Oct', 96, 1662.0], ['B', 'Nov', 78, 1254.0], ['B', 'Dec', 106, 1690.0], ['C', 'Jan', 42, 0.0], ['C', 'Feb', 0, 1967.0], ['C', 'Mar', 68, 0.0], ['C', 'Apr', 75, 0.0], ['C', 'May', 72, 0.0], ['C', 'Jun', 0, 1642.0], ['C', 'Jul', 0, 2489.0], ['C', 'Aug', 0, 2496.0], ['C', 'Sep', 33, 0.0], ['C', 'Oct', 0, 2421.0], ['C', 'Nov', 0, 963.0], ['C', 'Dec', 0, 1998.0]]


In [43]:
LB_Output = pd.DataFrame(LB_Output, columns=['State','Month','Optimal Staff_LBCase','Out_Appl_LBCase'])
LB_Output


,State,Month,Optimal Staff_LBCase,Out_Appl_LBCase
0,A,Jan,187,0.0
1,A,Feb,131,1463.0
2,A,Mar,212,0.0
3,A,Apr,77,0.0
4,A,May,71,0.0
5,A,Jun,61,682.0
6,A,Jul,156,1600.0
7,A,Aug,91,1011.0
8,A,Sep,134,0.0
9,A,Oct,95,1058.0


In [44]:
# Writing the results in to an Excel sheet
# Writing the output data into an Excel sheet named "Staffing_Data.xlsx" workbook

from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output'
LB_Output.to_excel(writer, sheet_name='Output_Staff_Planning_LB')


# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


In [45]:
# Including worst case results to overall dataframe for comparison
from statistics import mean
lb_final_output = []
lb_appl_cost = 0
lb_out_appl = 0
for l in model.location:
    for m in model.months:
            
            lb_cost_per_appl=((model_lb.y[l,m].value*model_lb.outsourcecost[l,m])+(model_lb.x[l,m].value*model_lb.staffsalary[l,m]))/model_lb.d[l,m]
            lb_appl_cost+=lb_cost_per_appl
            lb_final_output.append(lb_cost_per_appl)
            lb_out_appl+= model_lb.y[l,m].value
            
lb_base_mean = mean(lb_final_output)
df_lb = pd.DataFrame(data={(round(lb_base_mean,1),round(lb_appl_cost,1),round(worst_case_cost,2),round(lb_out_appl,))},index=['Worst Case Scenario'],columns=['Avg_cost_per_appl','tot_cost_per_appl','Overall Cost_in $mio','Outsource_Appl_count'])
overall_df = overall_df.append(df_lb)
overall_df


,Avg_cost_per_appl,tot_cost_per_appl,Overall Cost_in $mio,Outsource_Appl_count
Base Scenario,155.9,5611.7,17.96,20749
Worst Case Scenario,169.6,6104.5,19.60,39808


<div class="alert alert-block alert-success">
    
### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [46]:
# Creating a model instance
model_ub = ConcreteModel()

COMMENTS :All sets & parameters remain same except availability as the Staff Availability % is a different reference for best case scenario. Recreating all for Upper Bound model : model_ub 

#### Define Pyomo sets and Parameters

In [47]:
# displaying the location set
model_ub.location = Set(initialize = state_name.tolist(),doc = 'States')
model_ub.location.display()

location : States
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    3 : {'A', 'B', 'C'}


In [48]:
# displaying the months set
model_ub.months = Set(initialize=month_name.tolist(), doc = 'Months')
model_ub.months.display()

months : Months
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   12 : {'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'}


**`COMMENTS :`** No change in Parameter logics for Demand , Staff Monthly Salary & Outsource Cost. Recreating the same for Upper Bound model :  model_ub 

In [49]:
model_ub.d = Param(model_ub.location, model_ub.months, initialize = Demand_data, doc = 'UB Demand')
model_ub.staffsalary = Param(model_ub.location,model_ub.months, initialize=StaffMsal, doc='UB Staff Monthly Salary')
model_ub.outsourcecost = Param(model_ub.location, model_ub.months, initialize = OutsourceRate, doc = 'UB Outsource Cost')

In [50]:
# Recalculating the Availability basis UB% 
UB_data = staff_data.set_index(['State','Month'])['UB'].to_dict()
model_ub.UB = Param(model_ub.location, model_ub.months, initialize = UB_data, doc = 'Upper Bound on Availability')
model_ub.UB.display()

UB : Upper Bound on Availability
    Size=36, Index=UB_index, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('A', 'Apr') :  0.85
    ('A', 'Aug') :  0.85
    ('A', 'Dec') :   0.7
    ('A', 'Feb') :  0.85
    ('A', 'Jan') :   0.9
    ('A', 'Jul') :  0.75
    ('A', 'Jun') :   0.8
    ('A', 'Mar') :   0.8
    ('A', 'May') :  0.85
    ('A', 'Nov') :  0.75
    ('A', 'Oct') :   0.8
    ('A', 'Sep') :   0.9
    ('B', 'Apr') :  0.85
    ('B', 'Aug') :  0.85
    ('B', 'Dec') :   0.7
    ('B', 'Feb') :  0.85
    ('B', 'Jan') :   0.9
    ('B', 'Jul') :  0.75
    ('B', 'Jun') :   0.8
    ('B', 'Mar') :   0.8
    ('B', 'May') :  0.85
    ('B', 'Nov') :  0.75
    ('B', 'Oct') :   0.8
    ('B', 'Sep') :   0.9
    ('C', 'Apr') :  0.85
    ('C', 'Aug') :  0.85
    ('C', 'Dec') :   0.7
    ('C', 'Feb') :  0.85
    ('C', 'Jan') :   0.9
    ('C', 'Jul') :  0.75
    ('C', 'Jun') :   0.8
    ('C', 'Mar') :   0.8
    ('C', 'May') :  0.85
    ('C', 'Nov') :  0.75
    ('C', 'Oct') :   0.

In [51]:
# Decision variables

model_ub.x = Var(model_ub.location, model_ub.months, domain=NonNegativeReals, doc='No. of FTE for UB')
model_ub.y = Var(model_ub.location, model_ub.months, domain=NonNegativeIntegers, doc='No. of Outsource Appl. for UB')

In [52]:
# Constraints
# Demand Constraint gets changed as the Availability % is to be replaced by UB % values. 

model_ub.dConst = ConstraintList()

for l in model_ub.location:
    for m in model_ub.months:
        model_ub.dConst.add(expr = model_ub.y[l,m] + model_ub.x[l,m]*(model_ub.UB[l,m]*40) == model_ub.d[l,m]) 


In [53]:
# No change in the logic for Outsourcing constraint by State A & B. Recreating the constraint for model_ub
model_ub.statelimit = ConstraintList()

for l in model_ub.location:
    for m in model_ub.months:
        if l == 'A':
            model_ub.statelimit.add(expr = model_ub.y[l,m] <= 0.3*model_ub.d[l,m])
        elif l == 'B':
            model_ub.statelimit.add(expr = model_ub.y[l,m] <= 0.4*model_ub.d[l,m])
        else:
            pass

In [54]:
# Objective function

# Objective function - the function is similar as in base scenario : 
# (per Appl cost for Outsource*Outsource Appl) + (Monthly Salary * no. of FTEs)
obj = 0
def obj_rule_ub(model_ub):
    global obj
    for l in model_ub.location:
        for m in model_ub.months:
            obj+=(model_ub.y[l,m]*model_ub.outsourcecost[l,m])+(model_ub.x[l,m]*model_ub.staffsalary[l,m])
    return obj

model_ub.result = Objective(rule=obj_rule_ub, sense= minimize)


In [55]:
# Invoking the solver

result_ub = SolverFactory('glpk').solve(model_ub)
result_ub.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.1537187099456787
# ----------------------------------------------------------
#   Solution Information
# -------------

In [56]:
# Print the value of the objective function
best_case_cost = int(model_ub.result())/10**6
best_case_cost

16.527535

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [57]:
# optimal number of staff members for the best case scenario
ub_staff_count = []
ub_avilb = 0

for l in model_ub.location:
    for m in model_ub.months:
        
        # optimal no. of staff members for Worst Case scenario
        ub_staff_count.append([l,m,round(model_ub.x[l,m].value,)])
        
ub_staff_count = pd.DataFrame(ub_staff_count, columns=['State','Month','Optimal Staff_BestCase'])
ub_staff_count


,State,Month,Optimal Staff_BestCase
0,A,Jan,146
1,A,Feb,143
2,A,Mar,186
3,A,Apr,68
4,A,May,59
5,A,Jun,71
6,A,Jul,178
7,A,Aug,99
8,A,Sep,104
9,A,Oct,110


#### 3.2.2 Percentage of outsourced applications

In [58]:
# write your code here

out_appl = []
for l in model_ub.location:
    for m in model_ub.months:
        
        # outsourced applications% = no. of Outsource applications(stored in variable y) divide by total demand(stored in variable d)*100
        outsource_percent = round(((model_ub.y[l,m].value)/(model_ub.d[l,m]))*100,1)  
        out_appl.append([l,m,outsource_percent])

out_appl = pd.DataFrame(out_appl, columns=['State','Month','Outsource Application %'])
out_appl #percentage of outsourced applications for best Case scenario

,State,Month,Outsource Application %
0,A,Jan,0.0
1,A,Feb,0.0
2,A,Mar,0.0
3,A,Apr,0.0
4,A,May,0.0
5,A,Jun,0.0
6,A,Jul,0.0
7,A,Aug,0.0
8,A,Sep,0.0
9,A,Oct,0.0


In [59]:
# Overall % of outsourced Applications 
out = 0
dd = 0

for l in model_ub.location:
    for m in model_ub.months:
        out+=(model_ub.y[l,m].value)
        dd+=(model_ub.d[l,m])
        
UB_appl_per = round(out/dd*100,1)
print("outsource Appl :",out)
print("total demand :",dd)
print("Overall percentage of Outsource Applications in Best Case Scenario is :",UB_appl_per,"%")

outsource Appl : 4652.0
total demand : 113294
Overall percentage of Outsource Applications in Best Case Scenario is : 4.1 %


#### 3.2.3 Average cost per application

In [60]:
# write your code here

from statistics import mean 
cost_per_appl = []

for l in model_ub.location:
    for m in model_ub.months:
        appl_cost = ((model_ub.y[l,m].value*model_ub.outsourcecost[l,m])+(model_ub.x[l,m].value*model_ub.staffsalary[l,m]))/model_ub.d[l,m]
        cost_per_appl.append(appl_cost)

cost = mean(cost_per_appl)
print("Average Cost per application for Best Case scenario :",round(cost,2))

Average Cost per application for Best Case scenario : 143.42


#### Writting Output  to Excel For BEST case

In [61]:
# Creating dataframe for the results
# optimal number of staff members for the Best case scenario
UB_Output = []
UB_avilb = 0

for l in model_ub.location:
    for m in model_ub.months:
        
        # optimal no. of staff members for Best Case scenario
        Staff_count_UB = (round(model_ub.x[l,m].value))
        
        # outsourced applications% = no. of Outsource applications(stored in variable y) divide by total demand(stored in variable d)*100
        Outsource_applications_UB = (model_ub.y[l,m].value)
        
                       
        UB_Output.append([l,m, Staff_count_UB, Outsource_applications_UB])
       
                
print(UB_Output)

[['A', 'Jan', 146, 0.0], ['A', 'Feb', 143, 0.0], ['A', 'Mar', 186, 0.0], ['A', 'Apr', 68, 0.0], ['A', 'May', 59, 0.0], ['A', 'Jun', 71, 0.0], ['A', 'Jul', 178, 0.0], ['A', 'Aug', 99, 0.0], ['A', 'Sep', 104, 0.0], ['A', 'Oct', 110, 0.0], ['A', 'Nov', 143, 0.0], ['A', 'Dec', 185, 0.0], ['B', 'Jan', 137, 0.0], ['B', 'Feb', 77, 0.0], ['B', 'Mar', 93, 0.0], ['B', 'Apr', 69, 0.0], ['B', 'May', 118, 0.0], ['B', 'Jun', 98, 0.0], ['B', 'Jul', 85, 1708.0], ['B', 'Aug', 77, 0.0], ['B', 'Sep', 125, 0.0], ['B', 'Oct', 130, 0.0], ['B', 'Nov', 63, 1254.0], ['B', 'Dec', 91, 1690.0], ['C', 'Jan', 32, 0.0], ['C', 'Feb', 58, 0.0], ['C', 'Mar', 59, 0.0], ['C', 'Apr', 66, 0.0], ['C', 'May', 60, 0.0], ['C', 'Jun', 51, 0.0], ['C', 'Jul', 83, 0.0], ['C', 'Aug', 73, 0.0], ['C', 'Sep', 26, 0.0], ['C', 'Oct', 76, 0.0], ['C', 'Nov', 32, 0.0], ['C', 'Dec', 71, 0.0]]


In [62]:
UB_Output = pd.DataFrame(UB_Output, columns=['State','Month','Optimal Staff_UBCase','Out_Appl_UBCase'])
UB_Output


,State,Month,Optimal Staff_UBCase,Out_Appl_UBCase
0,A,Jan,146,0.0
1,A,Feb,143,0.0
2,A,Mar,186,0.0
3,A,Apr,68,0.0
4,A,May,59,0.0
5,A,Jun,71,0.0
6,A,Jul,178,0.0
7,A,Aug,99,0.0
8,A,Sep,104,0.0
9,A,Oct,110,0.0


In [63]:
# Writing the results in to an Excel sheet
# Writing the output data into an Excel sheet named "Staffing_Data.xlsx" workbook


book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output'
UB_Output.to_excel(writer, sheet_name='Output_Staff_Planning_UB')


# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


In [64]:
# Including best case results to overall dataframe for comparison
from statistics import mean
ub_final_output = []
ub_appl_cost = 0
ub_out_appl = 0
for l in model.location:
    for m in model.months:
            
            ub_cost_per_appl=((model_ub.y[l,m].value*model_ub.outsourcecost[l,m])+(model_ub.x[l,m].value*model_ub.staffsalary[l,m]))/model_ub.d[l,m]
            ub_appl_cost+=ub_cost_per_appl
            ub_final_output.append(ub_cost_per_appl)
            ub_out_appl+= model_ub.y[l,m].value
            
ub_base_mean = mean(ub_final_output)
df_ub = pd.DataFrame(data={(round(ub_base_mean,1),round(ub_appl_cost,1),round(best_case_cost,2),round(ub_out_appl,))},index=['Best Case Scenario'],columns=['Avg_cost_per_appl','tot_cost_per_appl','Overall Cost_in $mio','Outsource_Appl_count'])
overall_df = overall_df.append(df_ub)
overall_df


,Avg_cost_per_appl,tot_cost_per_appl,Overall Cost_in $mio,Outsource_Appl_count
Base Scenario,155.9,5611.7,17.96,20749
Worst Case Scenario,169.6,6104.5,19.60,39808
Best Case Scenario,143.4,5163.0,16.53,4652


# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

***Answer for this question has been added in the report file.***